<a href="https://colab.research.google.com/github/moodlep/MLC_A3C/blob/main/a3c_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 9.1 MB/s 


In [2]:
# Clone A3C repo: need PAT
! git clone https://xxxx@github.com/moodlep/MLC_A3C.git

Cloning into 'MLC_A3C'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 37 (delta 8), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [3]:
%cd /content/MLC_A3C
!pip install -e .

/content/MLC_A3C
Obtaining file:///content/MLC_A3C
  Running setup.py develop for a3c


In [4]:
import os
import Box2D
import pyglet
import imageio
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'


In [5]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


In [9]:
import torch.multiprocessing as mp
from actor_critic_networks import Critic, Policy
from shared_adam import SharedAdam
from actor_critic_worker import ActorCriticWorker


# if __name__ == '__main__':
#     mp.freeze_support()  # https://github.com/pytorch/pytorch/issues/5858

# Create a dummy env
env = gym.make("LunarLander-v2")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n


global_critic = Critic(state_dim)
global_actor = Policy(state_dim,action_dim)
global_critic.share_memory()
global_actor.share_memory()
n_workers = 10
env_name = "LunarLander-v2"

global_opt = SharedAdam(list(list(global_critic.parameters()) + list(global_actor.parameters())))
global_ctr = mp.Value('i',0)  # T - the global step counter
lock = mp.Lock()  # send to worker when it needs to update global counter

# pr = [ ActorCriticWorker(env_name,global_critic,global_actor,global_opt,global_ctr,lock) for _ in range(mp.cpu_count())]
pr = [ ActorCriticWorker(env_name,global_critic,global_actor,global_opt,global_ctr,lock) for _ in range(1)]

for p in pr:
    print(type(p))
    p.start()  # start each ActorCriticWorker process

for p in pr:
    p.join()  # wait for process to finish

<class 'actor_critic_worker.ActorCriticWorker'>


Process ActorCriticWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/content/MLC_A3C/actor_critic_worker.py", line 47, in run
    action = self.actor.get_action(torch.tensor(state, dtype=torch.double))
  File "/content/MLC_A3C/actor_critic_networks.py", line 27, in get_action
    policy = self.forward(state)
  File "/content/MLC_A3C/actor_critic_networks.py", line 20, in forward
    q = F.leaky_relu(self.l1(state))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/linear.py", line 96, in forward
    return F.linear(input, self.weight, self.bias)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py", line 1847, in linear
    return torch._C._nn.linear(input, weight, bias)
RuntimeError: expected scalar type Double but f